# Hyperparameter Tuning with the HParams Dashboard

## Setup

In [11]:
import tensorflow as tf
from tensorflow.keras import (
    datasets,
    Sequential,
    layers,
    callbacks
)
from tensorboard.plugins.hparams import api as hp 

In [49]:
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

In [2]:
!rm -rf logs

## Download the Data

In [4]:
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [37]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.2, 0.5))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

HP_METRIC = 'accuracy'

logdir = 'logs/hparam_tuning/'
with tf.summary.create_file_writer(logdir).as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(HP_METRIC, display_name='Accuracy')]
    )

## Adapt TensorFlow Runs to Log Hyperparameters and Metrics

In [43]:
def train_test_model(hparams):
    model = Sequential([
        layers.Flatten(),
        layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        layers.Dropout(hparams[HP_DROPOUT]),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(
        optimizer = hparams[HP_OPTIMIZER],
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
    
    cb_list = [
        callbacks.TensorBoard(logdir),
        hp.KerasCallback(logdir, hparams)
    ]
    
    model.fit(x_train, y_train, 
              epochs=5, 
              verbose=0,
              callbacks=cb_list)
    
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    
    return accuracy

In [48]:
!rm -rf logs

session_num = 0
for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer
            }
            run_name = 'run_%d' % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            train_test_model(hparams)
            session_num += 1

--- Starting trial: run_0
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}


2022-03-15 14:01:47.066140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 14:02:23.332900: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


--- Starting trial: run_1
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}


2022-03-15 14:02:24.617540: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 14:02:56.058740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


--- Starting trial: run_2
{'num_units': 16, 'dropout': 0.5, 'optimizer': 'adam'}


...

2022-03-15 14:04:42.472513: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 14:05:14.176483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


--- Starting trial: run_6
{'num_units': 32, 'dropout': 0.5, 'optimizer': 'adam'}


2022-03-15 14:05:15.393980: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 14:05:50.531743: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


--- Starting trial: run_7
{'num_units': 32, 'dropout': 0.5, 'optimizer': 'sgd'}


2022-03-15 14:05:51.850118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-15 14:06:23.199094: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


## Start Runs and Log Them All Under One Parent Directory